# 責任ある AI ダッシュボードを作成してモデルを評価する

機械学習モデルを比較して評価する場合は、パフォーマンス メトリック以外のものを確認したい場合があります。 Azure Machine Learning を使用すると、責任ある AI ダッシュボードを作成して、データのさまざまなコーホートに対するモデルのパフォーマンスを調べることができます。 

## データを準備する

責任ある AI ダッシュボードを作成するには、トレーニングおよびテスト データセットが必要です。Parquet ファイルとして保存され、データ資産として登録されます。

現在、データは CSV ファイルとして保存されています。 それらを Parquet ファイルに変換してみましょう。 


In [ ]:
## 続行する前に、次の点に注意してください。

このノートブックでコードを実行するには、**azureml-ai-ml** パッケージの最新バージョンが必要です。 次のセルを実行して、パッケージがインストールされていることを確認します。

> **注**:
> **azure-ai-ml** パッケージがインストールされていない場合は、`pip install azure-ai-ml` を実行してインストールします。

In [ ]:
## ワークスペースに接続する

必要な SDK パッケージがインストールされているため、ワークスペースに接続できます。

ワークスペースに接続するには、識別子パラメーター (サブスクリプション ID、リソース グループ名、ワークスペース名) が必要です。 リソース グループ名とワークスペース名は、既に入力されています。 コマンドを完了するには、サブスクリプション ID のみが必要です。

必要なパラメーターを見つけるには、スタジオの右上にあるサブスクリプションとワークスペース名をクリックします。 右側にペインが開きます。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> サブスクリプション ID をコピーし、**YOUR-SUBSCRIPTION-ID** をコピーした値に置き換えます。 </p>

## データ資産を作成する

責任ある AI ダッシュボードを作成するには、トレーニングおよびテスト データセットを **MLtable** データ資産として登録する必要があります。 MLtable データ資産は、前に作成した Parquet ファイルを参照します。

In [ ]:
## 責任ある AI ダッシュボードを作成するパイプラインを構築する

ダッシュボードを作成するには、Azure Machine Learning ワークスペースに既定で登録されている事前構築済みコンポーネントを使用してパイプラインを構築します。

### モデルを登録する

機械学習モデルは既にトレーニングされています。 このモデルでは、患者が糖尿病かどうかを予測します。 システム ファイルは、`model` フォルダーに保存されています。 

`model` フォルダーとその内容をポイントしてモデルを登録します。

In [ ]:
## パイプラインを構築する

責任ある AI ダッシュボードを作成するには、事前構築済みコンポーネントを使用してパイプラインを作成します。 使用するコンポーネントと、ダッシュボードに含める機能を選択できます。 エラー分析とモデルの解釈可能性を含むダッシュボードを作成します。 

責任ある AI 機能は別として、ダッシュボードを構築するためのパイプラインには、ダッシュボードを構築するための最初のコンポーネントと、生成されたすべての分析情報を収集するための最後のコンポーネントが含まれている必要があります。

In [ ]:
使用するすべてのコンポーネントを取得したら、パイプラインを構築し、適切な順序でコンポーネントを接続できます。

1. ダッシュボードを作成します。
1. エラー分析を追加します。
1. 説明を追加します。
1. すべての分析情報を収集し、ダッシュボードで視覚化します。

パイプラインが構築されたので、トレーニングおよびテスト データセットという 2 つの必要な入力を定義する必要があります。

In [ ]:
最後に、すべてをまとめます。パイプラインに入力を割り当て、ターゲット列 (予測ラベル) を設定します。

In [ ]:
## パイプラインを実行する

パイプラインが正常に構築されたら、それを送信できます。 次のコードでは、パイプラインを送信し、パイプラインの状態を確認します。 スタジオでパイプラインの状態を表示することもできます。 

パイプラインが完了したら、スタジオでダッシュボードを確認できます。 

In [ ]:
# Get handle to azureml registry for the RAI built in components
registry_name = "azureml"
ml_client_registry = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    registry_name=registry_name,
)
print(ml_client_registry)

### Register the model

A machine learning model has already been trained for you. The model predicts whether a patient has diabetes. All model files are stored in the `model` folder. 

Register the model by pointing to the `model` folder and its contents.

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="model",
    type=AssetTypes.MLFLOW_MODEL,
    name="local-mlflow-diabetes",
    description="Model created from local file.",
)
model = ml_client.models.create_or_update(file_model)

## Build the pipeline

To create the responsible AI dashboard, you'll create a pipeline using the prebuilt components. You can choose which components to use, and which features you want to include in your dashboard. You'll create a dashboard that includes error analysis and model interpretability. 

Next to the responsible AI features, a pipeline to build a dashboard needs to include a component at the start to construct the dashboard, and a component at the end to gather all generated insights.

In [ ]:
model_name = model.name
expected_model_id = f"{model_name}:1"
azureml_model_id = f"azureml:{expected_model_id}"

In [ ]:
label = "latest"

rai_constructor_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_constructor", label=label
)

# we get latest version and use the same version for all components
version = rai_constructor_component.version
print("The current version of RAI built-in components is: " + version)

rai_erroranalysis_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_erroranalysis", version=version
)

rai_explanation_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_explanation", version=version
)

rai_gather_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_gather", version=version
)

When you've retrieved all components you want to use, you can build the pipeline and connect the components in the appropriate order:

1. Construct the dashboard.
1. Add error analysis.
1. Add explanations.
1. Gather all insights and visualize them in the dashboard.

In [ ]:
from azure.ai.ml import Input, dsl
from azure.ai.ml.constants import AssetTypes

compute_name = "aml-cluster"

@dsl.pipeline(
    compute=compute_name,
    description="RAI insights on diabetes data",
    experiment_name=f"RAI_insights_{model_name}",
)
def rai_decision_pipeline(
    target_column_name, train_data, test_data
):
    # Initiate the RAIInsights
    create_rai_job = rai_constructor_component(
        title="RAI dashboard diabetes",
        task_type="classification",
        model_info=expected_model_id,
        model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=azureml_model_id),
        train_dataset=train_data,
        test_dataset=test_data,
        target_column_name=target_column_name,
    )
    create_rai_job.set_limits(timeout=30)

    # Add error analysis
    error_job = rai_erroranalysis_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
    )
    error_job.set_limits(timeout=10)

    # Add explanations
    explanation_job = rai_explanation_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
        comment="add explanation", 
    )
    explanation_job.set_limits(timeout=10)

    # Combine everything
    rai_gather_job = rai_gather_component(
        constructor=create_rai_job.outputs.rai_insights_dashboard,
        insight_3=error_job.outputs.error_analysis,
        insight_4=explanation_job.outputs.explanation,
    )
    rai_gather_job.set_limits(timeout=10)

    rai_gather_job.outputs.dashboard.mode = "upload"

    return {
        "dashboard": rai_gather_job.outputs.dashboard,
    }

Now the pipeline has been built, you need to define the two necessary inputs: the training and test dataset.

In [ ]:
from azure.ai.ml import Input
target_feature = "Diabetic"

diabetes_train_pq = Input(
    type="mltable",
    path=f"azureml:{input_train_data}:{data_version}",
    mode="download",
)
diabetes_test_pq = Input(
    type="mltable",
    path=f"azureml:{input_test_data}:{data_version}",
    mode="download",
)

Finally, we'll put everything together: assign the inputs to the pipeline and set the target column (the predicted label).

In [ ]:
import uuid
from azure.ai.ml import Output

# Pipeline to construct the RAI Insights
insights_pipeline_job = rai_decision_pipeline(
    target_column_name="Diabetic",
    train_data=diabetes_train_pq,
    test_data=diabetes_test_pq,
)

# Workaround to enable the download
rand_path = str(uuid.uuid4())
insights_pipeline_job.outputs.dashboard = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/dashboard/",
    mode="upload",
    type="uri_folder",
)


## Run the pipeline

When you've successfully built the pipeline, you can submit it. The following code will submit the pipeline and check the status of the pipeline. You can also view the pipeline's status in the studio. 

In [ ]:
from azure.ai.ml.entities import PipelineJob
from IPython.core.display import HTML
from IPython.display import display
import time

def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    print("Pipeline job can be accessed in the following URL:")
    display(HTML('<a href="{0}">{0}</a>'.format(created_job.studio_url)))

    while created_job.status not in [
        "Completed",
        "Failed",
        "Canceled",
        "NotResponding",
    ]:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == "Completed"
    return created_job


# This is the actual submission
insights_job = submit_and_wait(ml_client, insights_pipeline_job)

When the pipeline is completed, you can review the dashboard in the studio. 